In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
import torchvision
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from libs.code import *
from pytorch_lightning.loggers import TensorBoardLogger
from libs.VAE import *

In [ ]:

# Setting the seed
pl.seed_everything(42)
# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)


PATH_DST = 'dataset/all_labels.csv'
PATH_GDRIVE = ''
NUM_WORKERS = 4
BATCH_SIZE = 32
NUM_EPOCHS = 5
GPUS = 0

transform = transforms.Compose([
                                transforms.Resize((32,32)),
                                transforms.ToTensor(),
                                # torch.flatten # trasforma il tensore ad una dimensione
                                ])

dataset = TrashbinDataset(csv=PATH_DST, transform=transform)

# TODO: fixa la funzione!
dataset_train, dataset_test = split_into_train_and_test(dataset)
_, dataset_val = split_into_train_and_test(dataset)

train_loader = data.DataLoader(dataset_train, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
val_loader = data.DataLoader(dataset_val, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)
test_loader = data.DataLoader(dataset_test, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)

In [ ]:
def train(latent_dim, checkpoint_path, name):
    # Create a PyTorch Lightning trainer with the generation callback

    my_logger = TensorBoardLogger(save_dir=f"{checkpoint_path}_logger", name= f"{name}_{latent_dim}")

    trainer = pl.Trainer(default_root_dir=os.path.join(f"{checkpoint_path}_trainer", f"{name}_{latent_dim}"), 
                        gpus=1 if str(device).startswith("cuda") else 0, 
                        max_epochs=10, 
                        callbacks=[ModelCheckpoint(save_weights_only=True),
                                    GenerateCallback(get_train_images(8), every_n_epochs=2),
                                    LearningRateMonitor("epoch")]
                        ,logger=my_logger)
                        
    # trainer.logger._log_graph = True         # If True, we plot the computation graph in tensorboard
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need
    
    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(checkpoint_path, f"{name}_{latent_dim}.ckpt")
    if os.path.isfile(pretrained_filename):
        print("Found pretrained model, loading...*****")
        model = Autoencoder.load_from_checkpoint(pretrained_filename)
    else:
        model = Autoencoder(base_channel_size=32, latent_dim=latent_dim)
        trainer.fit(model, train_loader, val_loader)
    
    # Test best model on validation and test set
    val_result = trainer.test(model, test_dataloaders=val_loader, verbose=False)
    test_result = trainer.test(model, test_dataloaders=test_loader, verbose=False)
    result = {"test": test_result, "val": val_result}
    return model, result

In [ ]:
model_dict = {}
for latent_dim in [64, 128, 256, 384]:
    model_ld, result_ld = train(latent_dim, f"VAE_{latent_dim}", f"VAE_{latent_dim}")
    model_dict[latent_dim] = {"model": model_ld, "result": result_ld}